In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM
from sklearn.base import BaseEstimator, RegressorMixin
from keras.models import Sequential
from tensorflow.keras.models import Model
from keras.layers import Input, GRU, LSTM, Dense, Dropout, Concatenate, Reshape
from keras.optimizers import Adam
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import load_model

In [17]:
test_data = pd.read_csv('test_stn+ef_label.csv')
test_data.head()

,stn4contest,ef_datetime,v01,v02,v03,v04,v05,v06,v07,v08,v09,year,month,day,hour,class_interval
0,31,2023-05-01 12:00:00,68.0,46.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,12,NaN
1,31,2023-05-01 15:00:00,83.0,66.0,26.0,5.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,15,NaN
2,31,2023-05-01 18:00:00,17.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,18,NaN
3,31,2023-05-01 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,21,NaN
4,31,2023-05-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,2,0,NaN


In [22]:
features = test_data[['ef_datetime', 'stn4contest', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09', 'year', 'month', 'day', 'hour']]
target = test_data['class_interval']

features.set_index('ef_datetime', inplace=True)

# 데이터 정렬
features = features.sort_index()

# 데이터 정규화
scaler_features = MinMaxScaler(feature_range=(0, 1))

test_features_scaled = scaler_features.fit_transform(features)

# NaN 값을 0으로 대체 (혹은 다른 값으로 처리)
test_features_scaled = np.nan_to_num(test_features_scaled)

# 입력 데이터를 GRU 레이어에 맞게 재구성
X_test = test_features_scaled.reshape((test_features_scaled.shape[0], test_features_scaled.shape[1], 1))

In [23]:
# 저장된 모델 로드
model = load_model('gru20.h5')

# 예측
test_predict = model.predict(X_test)

# 예측 결과를 데이터프레임으로 변환
predicted_df = pd.DataFrame(test_predict, index=features.index, columns=['Predicted vv'])

# 예측 결과 출력
predicted_df

204/204 [==============================] - 4s 8ms/step


,Predicted vv
ef_datetime,
2023-05-01 12:00:00,0.314307
2023-05-01 12:00:00,0.162190
2023-05-01 12:00:00,-0.004495
2023-05-01 12:00:00,0.064912
2023-05-01 12:00:00,0.110233
...,...
2023-10-10 21:00:00,0.135040
2023-10-10 21:00:00,-0.052316
2023-10-10 21:00:00,-0.135212


In [34]:
def get_class_interval(rainfall):
    if rainfall < 0.1:
        return 0
    elif 0.1 <= rainfall < 0.2:
        return 1
    elif 0.2 <= rainfall < 0.5:
        return 2
    elif 0.5 <= rainfall < 1.0:
        return 3
    elif 1.0 <= rainfall < 2.0:
        return 4
    elif 2.0 <= rainfall < 5.0:
        return 5
    elif 5.0 <= rainfall < 10.0:
        return 6
    elif 10.0 <= rainfall < 20.0:
        return 7
    elif 20.0 <= rainfall < 30.0:
        return 8
    else:
        return 9

In [26]:
# 'Predicted vv' 열에 변환 함수 적용
predicted_df['class_interval'] = predicted_df['Predicted vv'].apply(get_class_interval)

In [28]:
predicted_df['class_interval'].value_counts()

class_interval
0    2644
1    1232
2     863
3     442
5     395
4     350
6     304
7     194
8      76
Name: count, dtype: int64

In [38]:
# 'Predicted vv' 열에 변환 함수 적용하여 클래스 구간 계산
predicted_df['Class Interval'] = predicted_df['Predicted vv'].apply(get_class_interval)

# 원래의 test_data에 'Class Interval' 열 추가
test_data['class_interval'] = predicted_df['Class Interval'].values

In [40]:
test_data = test_data.drop(['Class Interval'], axis=1)

test_data.head()

,stn4contest,ef_datetime,v01,v02,v03,v04,v05,v06,v07,v08,v09,year,month,day,hour,class_interval
0,31,2023-05-01 12:00:00,68.0,46.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,12,2
1,31,2023-05-01 15:00:00,83.0,66.0,26.0,5.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,15,1
2,31,2023-05-01 18:00:00,17.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,18,0
3,31,2023-05-01 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,1,21,0
4,31,2023-05-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,5,2,0,1


In [45]:
submit.head()

,Unnamed: 0,rainfall_test.fc_year,rainfall_test.fc_month,rainfall_test.fc_day,rainfall_test.fc_hour,rainfall_test.stn4contest,rainfall_test.dh,rainfall_test.ef_year,rainfall_test.ef_month,rainfall_test.ef_day,...,rainfall_test.v01,rainfall_test.v02,rainfall_test.v03,rainfall_test.v04,rainfall_test.v05,rainfall_test.v06,rainfall_test.v07,rainfall_test.v08,rainfall_test.v09,rainfall_test.class_interval
0,1,D,5,1,9,STN031,3,D,5,1,...,68,46,13,1,0,0,0,0,0,NaN
1,2,D,5,1,9,STN031,6,D,5,1,...,83,66,26,5,0,0,0,0,0,NaN
2,3,D,5,1,9,STN031,9,D,5,1,...,17,6,0,0,0,0,0,0,0,NaN
3,4,D,5,1,9,STN031,12,D,5,1,...,0,0,0,0,0,0,0,0,0,NaN
4,5,D,5,1,9,STN031,15,D,5,2,...,0,0,0,0,0,0,0,0,0,NaN


In [48]:
submit = pd.read_csv("rainfall_test2.csv")
realsubmit = pd.read_csv("rainfall_test.csv")
prediction_test = test_data.copy()

#submit-prediction 지점,date 맞추기
prediction_test['ef_datetime'] = pd.to_datetime(prediction_test['ef_datetime'])
submit['ef_year_temp'] = submit['ef_year'].replace({'D': '2023'})
submit['ef_datetime'] = pd.to_datetime(submit[['ef_year_temp', 'ef_month', 'ef_day', 'ef_hour']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H')
stn_mapping = {'STN031': 31, 'STN032': 32, 'STN033': 33, 'STN034': 34, 'STN035': 35}
submit['stn4contest'] = submit['stn4contest'].replace(stn_mapping)

#test_data['class_interval'] = y_pred2
#test_data['class_interval'] = test_data['class_interval'].apply(classify_class)

    
merged = submit.merge(prediction_test[['ef_datetime', 'stn4contest', 'class_interval']], 
                      on=['ef_datetime', 'stn4contest'], 
                      how='left', 
                      suffixes=('', '_pred'))
merged['class_interval_pred']
    
#-999 index 저장
idx999 = realsubmit[realsubmit['rainfall_test.class_interval'].notnull()].index
merged.loc[idx999, 'class_interval_pred'] = -999

# 제출용 데이터에 class_interval 채우기
realsubmit['rainfall_test.class_interval'] = merged['class_interval_pred']

#저장
realsubmit.to_csv("240494.csv")  
print(realsubmit.shape)
realsubmit.head()

(122000, 21)


,Unnamed: 0,rainfall_test.fc_year,rainfall_test.fc_month,rainfall_test.fc_day,rainfall_test.fc_hour,rainfall_test.stn4contest,rainfall_test.dh,rainfall_test.ef_year,rainfall_test.ef_month,rainfall_test.ef_day,...,rainfall_test.v01,rainfall_test.v02,rainfall_test.v03,rainfall_test.v04,rainfall_test.v05,rainfall_test.v06,rainfall_test.v07,rainfall_test.v08,rainfall_test.v09,rainfall_test.class_interval
0,1,D,5,1,9,STN031,3,D,5,1,...,68,46,13,1,0,0,0,0,0,2
1,2,D,5,1,9,STN031,6,D,5,1,...,83,66,26,5,0,0,0,0,0,1
2,3,D,5,1,9,STN031,9,D,5,1,...,17,6,0,0,0,0,0,0,0,0
3,4,D,5,1,9,STN031,12,D,5,1,...,0,0,0,0,0,0,0,0,0,0
4,5,D,5,1,9,STN031,15,D,5,2,...,0,0,0,0,0,0,0,0,0,1


In [49]:
realsubmit['rainfall_test.class_interval'].unique()

array([   2,    1,    0, -999,    5,    3,    4,    6,    7,    8],
      dtype=int64)

-----

## GRU(look_back=48)

In [60]:
data = pd.read_csv('use_this.csv')

In [62]:
features = data[['stn4contest', 'year', 'month', 'day', 'hour', 'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']]
target = data['vv']

In [64]:
# 스케일링
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features.drop(columns=['stn4contest']))
features_scaled_df = pd.DataFrame(features_scaled, columns=features.columns[1:])
features_scaled_df['stn4contest'] = features['stn4contest'].values
features_scaled_df['vv'] = target.values

In [65]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 시퀀스 생성 함수
def create_sequences(data, target, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data.iloc[i:i + lookback].values)
        y.append(target.iloc[i + lookback])
    return np.array(X), np.array(y)

In [66]:
lookbacks = [48]
results = {}
predictions = {}

# 최적의 하이퍼파라미터
best_params = {'units3': 32, 'units2': 64, 'units1': 32, 'learning_rate': 0.001, 'dropout_rate': 0.2}

for lookback in lookbacks:
    X_train, y_train = [], []
    X_test, y_test = [], []
    
    for stn in features_scaled_df['stn4contest'].unique():
        stn_data = features_scaled_df[features_scaled_df['stn4contest'] == stn]
        stn_target = stn_data['vv']
        stn_data = stn_data.drop(columns=['stn4contest', 'vv'])
        
        X, y = create_sequences(stn_data, stn_target, lookback)
        
        if stn <= 16:
            X_train.append(X)
            y_train.append(y)
        else:
            X_test.append(X)
            y_test.append(y)
    
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)
    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)
    
    # 최적의 하이퍼파라미터로 모델 학습
    model = Sequential()
    model.add(Input(shape=(lookback, X_train.shape[2])))
    model.add(GRU(units=best_params['units1'], return_sequences=True))
    model.add(Dropout(best_params['dropout_rate']))
    model.add(GRU(units=best_params['units2'], return_sequences=True))
    model.add(Dropout(best_params['dropout_rate']))
    model.add(GRU(units=best_params['units3']))
    model.add(Dropout(best_params['dropout_rate']))
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=best_params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
    
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping, model_checkpoint])
    
    # 모델 평가 및 예측 값 저장
    y_pred = model.predict(X_test)
    predictions[lookback] = (y_test, y_pred)
    loss = model.evaluate(X_test, y_test, verbose=0)
    results[lookback] = loss

# 성능 지표 계산 및 출력
print("Model Performance Metrics by Lookback:")
print(f"{'Lookback':<15}{'RMSE':<10}{'MAE':<10}{'MSE':<10}{'R²':<10}")
print(f"{'-'*45}")

for lookback, (y_test, y_pred) in predictions.items():
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{lookback:<15}{rmse:<10.4f}{mae:<10.4f}{mse:<10.4f}{r2:<10.4f}")

362/362 [==============================] - 6s 15ms/step
Model Performance Metrics by Lookback:
Lookback       RMSE      MAE       MSE       R²        
---------------------------------------------
48             4.5114    1.4943    20.3530   -0.0022   
